In [100]:
!pip install pytesseract easyocr opencv-python numpy pandas scikit-learn tensorflow keras

In [101]:
import cv2
import pytesseract
import numpy as np
import pandas as pd
import re
import string
from PIL import Image


In [106]:
df = pd.read_csv("angel1out.csv")
  # Contains 'Description' and 'Genre' columns 
df.drop(columns=['Unnamed: 0', 'Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 6', 'Unnamed: 4'], inplace=True)
df

,title,descripton,genre
0,Sapiens: A Brief History of Humankind,"100,000 years ago, at least six human species ...",history
1,"Guns, Germs, and Steel: The Fates of Human Soc...","""Diamond has written a book of remarkable scop...",history
2,A People's History of the United States,"In the book, Zinn presented a different side o...",history
3,"The Devil in the White City: Murder, Magic, an...",Author Erik Larson imbues the incredible event...,history
4,The Diary of a Young Girl,Discovered in the attic in which she spent the...,history
...,...,...,...
1905,The Distant Hours,A man obsessed with vengeance seeks retributio...,fiction
1906,The House at Riverton,A young woman navigates love and societal expe...,fiction
1907,The Summer I Turned Pretty,"A young orphan girl discovers a hidden, neglec...",fiction
1908,We'll Always Have Summer,A wealthy and enigmatic man throws lavish part...,fiction


In [108]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910 entries, 0 to 1909
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       1910 non-null   object
 1   descripton  1910 non-null   object
 2   genre       1910 non-null   object
dtypes: object(3)
memory usage: 44.9+ KB


In [112]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['genre_Label'] = encoder.fit_transform(df['genre'])
genre_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Store genre mapping
print(genre_mapping)

{'fantasy': 0, 'fiction': 1, 'history': 2, 'horror': 3, 'psychology': 4, 'romance': 5, 'science': 6, 'science_fiction': 7, 'sports': 8, 'thriller': 9, 'travel': 10}


In [114]:
df

,title,descripton,genre,genre_Label
0,Sapiens: A Brief History of Humankind,"100,000 years ago, at least six human species ...",history,2
1,"Guns, Germs, and Steel: The Fates of Human Soc...","""Diamond has written a book of remarkable scop...",history,2
2,A People's History of the United States,"In the book, Zinn presented a different side o...",history,2
3,"The Devil in the White City: Murder, Magic, an...",Author Erik Larson imbues the incredible event...,history,2
4,The Diary of a Young Girl,Discovered in the attic in which she spent the...,history,2
...,...,...,...,...
1905,The Distant Hours,A man obsessed with vengeance seeks retributio...,fiction,1
1906,The House at Riverton,A young woman navigates love and societal expe...,fiction,1
1907,The Summer I Turned Pretty,"A young orphan girl discovers a hidden, neglec...",fiction,1
1908,We'll Always Have Summer,A wealthy and enigmatic man throws lavish part...,fiction,1


In [116]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('all')

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)

df['Cleaned_Description'] = df['descripton'].apply(preprocess_text)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_

In [117]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Cleaned_Description']).toarray()
y = df['genre_Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [120]:
df

,title,descripton,genre,genre_Label,Cleaned_Description
0,Sapiens: A Brief History of Humankind,"100,000 years ago, at least six human species ...",history,2,100 000 years ago least six human species inha...
1,"Guns, Germs, and Steel: The Fates of Human Soc...","""Diamond has written a book of remarkable scop...",history,2,diamond written book remarkable scope one impo...
2,A People's History of the United States,"In the book, Zinn presented a different side o...",history,2,book zinn presented different side history tra...
3,"The Devil in the White City: Murder, Magic, an...",Author Erik Larson imbues the incredible event...,history,2,author erik larson imbues incredible events su...
4,The Diary of a Young Girl,Discovered in the attic in which she spent the...,history,2,discovered attic spent last years life anne fr...
...,...,...,...,...,...
1905,The Distant Hours,A man obsessed with vengeance seeks retributio...,fiction,1,man obsessed vengeance seeks retribution white...
1906,The House at Riverton,A young woman navigates love and societal expe...,fiction,1,young woman navigates love societal expectatio...
1907,The Summer I Turned Pretty,"A young orphan girl discovers a hidden, neglec...",fiction,1,young orphan girl discovers hidden neglected g...
1908,We'll Always Have Summer,A wealthy and enigmatic man throws lavish part...,fiction,1,wealthy enigmatic man throws lavish parties 19...


In [122]:
# Step 7: Train the model
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train, y_train)


# Step 8: Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
#print("\nClassification Report:\n", classification_report(y_test, y_pred))
#print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Step 9: Save the model and vectorizer
#joblib.dump(model, "genre_prediction_model.pkl")
#joblib.dump(tfidf, "tfidf_vectorizer.pkl")
#joblib.dump(label_encoder, "genre_label_encoder.pkl")

Accuracy: 0.7905759162303665


C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [124]:
!pip install pytesseract
!pip install pillow
!pip install scikit-learn


In [125]:
import pytesseract
from PIL import Image
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [126]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.


In [135]:
import easyocr
from PIL import Image

# Create an EasyOCR reader
reader = easyocr.Reader(['en'])  # 'en' stands for English. You can add other languages as needed

# Load the image
image_path = r"C:\Users\ASUS\Downloads\81iP21l3QaL._SL1500_.jpg"
image = Image.open(image_path)

# Perform OCR and extract text
result = reader.readtext(image_path)

# Print the extracted text
for detection in result:
    print(detection[1])  # The second element contains the recognized text


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


It is 326 BC, and Alexander;
the barbarian
of Macedonia,
3
has descended upon Bharatvarsha with
a multinational horde of Yavanas,
8
Pahlavas, Shakas, and Bahlikas.
As the invader advances relentlessly and wins bloody battles in quick
8
succession,as local rulers fall over each other to shakehands with the
THE BOY FROM
enemy,and as the students of Takshashila University breakinto open
revolt, one young man is faced with a terrifying choice, a choice that
PATALIPUTRA
threatens to tear his carefully constructed world apart. For
the
from Pataliputra; the
who was once
a reckless and
1
carefree aristocrat; but who has now been forced to become a man
with a purpose
tto
fight forhonourand love: 
With a sweeping narrative and interesting everyday characters like
the smelly; old dhaba owner Tanku; Philotas; the unlucky Greek
soldier; the no-nonsense medical student Radha; Pandi; the hard
drinking mercenary; and the lovely Devika, The Boy from Pataliputra
is not just the mesmerizing
of
young man$

In [137]:
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'])  # Initialize EasyOCR reader for English
    results = reader.readtext(image_path, detail=0)  # Extract text from image
    return " ".join(results)  # Return extracted text as a single string

def predict_genre(image_path):
    text = extract_text_easyocr(image_path)
    processed_text = preprocess_text(text)

    # Machine Learning Prediction
    text_tfidf = vectorizer.transform([processed_text]).toarray()
    ml_prediction = model.predict(text_tfidf)
    ml_genre = encoder.inverse_transform(ml_prediction)[0]
    
    return {"Genre": ml_genre}

# Test with a new image
image_path = r"C:\Users\ASUS\Downloads\81iP21l3QaL._SL1500_.jpg"
prediction = predict_genre(image_path)
print(prediction)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


{'Genre': 'fiction'}
